In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import math


In [3]:
X=torch.randn(128,64,512) #batch,time,dimension
print(X.shape)

torch.Size([128, 64, 512])


In [4]:
d_model=512
n_head=8

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.w_q=nn.Linear(d_model,d_model)
        self.w_k=nn.Linear(d_model,d_model)
        self.w_v=nn.Linear(d_model,d_model)
        self.w_o=nn.Linear(d_model,d_model)
        
        self.softmax=nn.Softmax(dim=-1)
        
    def forward(self, q,k,v):
        batch,time,dimension=q.shape
        n_d=self.d_model//self.num_heads
        
        q=self.w_q(q)
        k=self.w_k(k)
        v=self.w_v(v)
        
        q=q.view(batch,time,self.num_heads,n_d).permute(0,2,1,3)
        k=k.view(batch,time,self.num_heads,n_d).permute(0,2,1,3)    
        v=v.view(batch,time,self.num_heads,n_d).permute(0,2,1,3)
        
        score=q@k.transpose(2,3)/math.sqrt(n_d)
        mask=torch.tril(torch.ones(time,time,dtype=bool))
        score=score.masked_fill_(mask==0,float("-inf"))
        
        attention=self.softmax(score)
        output=attention@v
        output=output.permute(0,2,1,3).contiguous().view(batch,time,dimension)
        output=self.w_o(output)
        return output

In [5]:
import torch
import torch.nn as nn
import math

In [6]:
d_model=512
n_head=8


In [ ]:
class MultiHeadAttentino(nn.Module):
    def __init__(self,d_model,num_heads):
        super(MultiHeadAttentino,self).__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.head_dim=self.d_model//self.num_heads
        
        self.query_linear=nn.Linear(d_model,d_model)
        self.key_linear=nn.Linear(d_model,d_model)
        self.value_linear=nn.Linear(d_model,d_model)
        self.output_linear=nn.Linear(d_model,d_model)
        
        self.softmax=nn.Softmax(dim=-1)
        
    def forward(self,query,key,value):
        batch_size, seq_len,_=query.shape   
        query=self.query_linear(query)
        key=self.key_linear(key)
        value=self.value_linear(value)
        
        query=query.view(batch_size,seq_len,self.num_heads,self.head_dim).permute(0,2,1,3)
        key=key.view(batch_size,seq_len,self.num_heads,self.head_dim).permute(0,2,1,3)
        value=value.view(batch_size,seq_len,self.num_heads,self.head_dim).permute(0,2,1,3)
        
        attention_scores=torch.matmul(query,key.transpose(2,3))/math.sqrt(self.head_dim)
        
        mask=torch.tril(torch.ones(seq_len,seq_len,dtype=bool))
        attention_scores=attention_scores.masked_fill_(mask==0,float("-inf"))
        
        attention=self.softmax(attention_scores)
            
        
    
        output=torch.matmul(attention,value)
        output=output.permute(0,2,1,3).contiguous().view(batch_size,seq_len,self.d_model)
        output=self.output_linear(output)
        return output
        